### データインポート

In [1]:

import pandas as pd
from logic.factory_manage.sql import load_data_from_sqlite

# CSVファイル読み込み（パスは適宜変更）
path = "/work/app/data/factory_manage/weight_data.db"
df = load_data_from_sqlite()
df["伝票日付"].max()

Timestamp('2025-06-19 00:00:00')

In [2]:
from utils.get_holydays import get_japanese_holidays
hol_max = df["伝票日付"].max()
hol_min = df["伝票日付"].min()
holiday = get_japanese_holidays(hol_min, hol_max)
holiday[:5]

['2024-05-03', '2024-05-04', '2024-05-05', '2024-05-06', '2024-07-15']

In [3]:
from logic.factory_manage.predict_model_v3 import generate_features, train_and_predict
df_feat, df_pivot = generate_features(df,holidays=holiday)

### 旧予測モデル

In [ ]:
from logic.factory_manage.predict_model_ver2 import train_and_predict_with_holiday
import pandas as pd
from datetime import timedelta


In [ ]:

roop_start_date = pd.to_datetime("2025-03-01")
roop_end_date = pd.to_datetime("2025-05-31")

results = []

for current_start in pd.date_range(start=roop_start_date, end=roop_end_date):
    start_dt = current_start
    end_dt = start_dt  # 同じ日に設定

    filtered_df = df[df['伝票日付'] < start_dt]

    if filtered_df.empty:
        print(f"Warning: No training data available before {start_dt.strftime('%Y-%m-%d')}, skipping.")
        continue

    print(f"date {filtered_df['伝票日付'].min()} to {filtered_df['伝票日付'].max()} with {len(filtered_df)} records")
    print(f"predicting for {start_dt.strftime('%Y-%m-%d')}")

    pred_df = train_and_predict_with_holiday(
        filtered_df,
        start_dt.strftime("%Y-%m-%d"),
        end_dt.strftime("%Y-%m-%d"),
        holiday
    )

    # 予測結果に予測対象日カラムを追加
    pred_df['予測対象日'] = start_dt.strftime('%Y-%m-%d')

    results.append(pred_df)

if results:
    all_results_df = pd.concat(results, ignore_index=True)
    all_results_df.to_csv("predictions_.csv", index=False)
    print("Prediction results saved to predictions_202505.csv")
else:
    print("No prediction results to save.")


### 真予測モデル

In [ ]:
from logic.factory_manage.predict_model_ver2 import train_and_predict_fix
import pandas as pd
from datetime import timedelta


In [ ]:

roop_start_date = pd.to_datetime("2025-03-01")
roop_end_date = pd.to_datetime("2025-05-31")

results = []

for current_start in pd.date_range(start=roop_start_date, end=roop_end_date):
    start_dt = current_start
    end_dt = start_dt  # 同じ日に設定

    filtered_df = df[df['伝票日付'] < start_dt]

    if filtered_df.empty:
        print(f"Warning: No training data available before {start_dt.strftime('%Y-%m-%d')}, skipping.")
        continue

    print(f"date {filtered_df['伝票日付'].min()} to {filtered_df['伝票日付'].max()} with {len(filtered_df)} records")
    print(f"predicting for {start_dt.strftime('%Y-%m-%d')}")

    pred_df = train_and_predict_fix(
        filtered_df,
        start_dt.strftime("%Y-%m-%d"),
        end_dt.strftime("%Y-%m-%d"),
        holiday
    )

    # 予測結果に予測対象日カラムを追加
    pred_df['予測対象日'] = start_dt.strftime('%Y-%m-%d')

    results.append(pred_df)

if results:
    all_results_df = pd.concat(results, ignore_index=True)
    all_results_df.to_csv("predictions_.csv", index=False)
    print("Prediction results saved to predictions_202505.csv")
else:
    print("No prediction results to save.")


## モデル改良版

In [6]:
from logic.factory_manage.predict_model_v3 import generate_features, train_and_predict
import pandas as pd
from datetime import timedelta


In [ ]:

roop_start_date = pd.to_datetime("2025-03-01")
roop_end_date = pd.to_datetime("2025-05-31")

results = []

for current_start in pd.date_range(start=roop_start_date, end=roop_end_date):
    start_dt = current_start
    end_dt = start_dt  # 同じ日に設定

    filtered_df = df[df['伝票日付'] < start_dt]

    if filtered_df.empty:
        print(f"Warning: No training data available before {start_dt.strftime('%Y-%m-%d')}, skipping.")
        continue

    print(f"date {filtered_df['伝票日付'].min()} to {filtered_df['伝票日付'].max()} with {len(filtered_df)} records")
    print(f"predicting for {start_dt.strftime('%Y-%m-%d')}")

    pred_df = train_and_predict(
        df_feat,
        df_pivot,
        # filtered_df,
        start_dt.strftime("%Y-%m-%d"),
        end_dt.strftime("%Y-%m-%d"),
        holidays=holiday,
    )

    # 予測結果に予測対象日カラムを追加
    pred_df['予測対象日'] = start_dt.strftime('%Y-%m-%d')

    results.append(pred_df)

if results:
    all_results_df = pd.concat(results, ignore_index=True)
    all_results_df.to_csv("predictions_.csv", index=False)
    print("Prediction results saved to predictions_202505.csv")
else:
    print("No prediction results to save.")


## モデル改良版/リークなし

In [14]:

from logic.factory_manage.predict_model_v4 import generate_features, run_fully_walkforward_pipeline
import pandas as pd
from datetime import timedelta

df_feat, df_pivot = generate_features(df,holidays=holiday)

# print(f"date {df['伝票日付'].min()} to {df['伝票日付'].max()} with {len(df)} records")
# print(f"predicting for {start_dt.strftime('%Y-%m-%d')}")

pred_df = run_fully_walkforward_pipeline(
    df_feat,
    df_pivot,
)



🕒 Processing 2024-05-13 ...
  📦 Training for 混合廃棄物A
  📦 Training for 混合廃棄物B
🕒 Processing 2024-05-14 ...
  📦 Training for 混合廃棄物A
  📦 Training for 混合廃棄物B
🕒 Processing 2024-05-15 ...
  📦 Training for 混合廃棄物A
  📦 Training for 混合廃棄物B
🕒 Processing 2024-05-16 ...
  📦 Training for 混合廃棄物A
  📦 Training for 混合廃棄物B
🕒 Processing 2024-05-17 ...
  📦 Training for 混合廃棄物A
  📦 Training for 混合廃棄物B
🕒 Processing 2024-05-18 ...
  📦 Training for 混合廃棄物A
  📦 Training for 混合廃棄物B
🕒 Processing 2024-05-19 ...
  📦 Training for 混合廃棄物A
  📦 Training for 混合廃棄物B
🕒 Processing 2024-05-20 ...
  📦 Training for 混合廃棄物A
  📦 Training for 混合廃棄物B
🕒 Processing 2024-05-21 ...
  📦 Training for 混合廃棄物A
  📦 Training for 混合廃棄物B
🕒 Processing 2024-05-22 ...
  📦 Training for 混合廃棄物A
  📦 Training for 混合廃棄物B
🕒 Processing 2024-05-23 ...
  📦 Training for 混合廃棄物A
  📦 Training for 混合廃棄物B
🕒 Processing 2024-05-24 ...
  📦 Training for 混合廃棄物A
  📦 Training for 混合廃棄物B
🕒 Processing 2024-05-25 ...
  📦 Training for 混合廃棄物A
  📦 Training for 混合廃棄物B
🕒 Processing

## モデル改良版/リークなし/データ強化版

### データ作成

In [ ]:
import pandas as pd
from logic.factory_manage.sql import load_data_from_sqlite

# Read the CSV file
path_2020 = "/work/app/data/input/2020顧客.csv"
path_2021 = "/work/app/data/input/2021顧客.csv"
path_2022 = "/work/app/data/input/2022顧客.csv"
path_2023 = "/work/app/data/input/2023_all.csv"
path_2024 = "/work/app/data/input/20240501-20250422.csv"


# CSV読み込み
df_2021 = pd.read_csv(path_2021, encoding="utf-8")[["伝票日付", "正味重量", "商品"]]
df_2022 = pd.read_csv(path_2022, encoding="utf-8")[["伝票日付", "正味重量", "商品"]]
df_2023 = pd.read_csv(path_2023, encoding="utf-8")[["伝票日付", "正味重量", "商品"]]
df_2024 = pd.read_csv(path_2024, encoding="utf-8")[["伝票日付", "正味重量", "品名"]]


# カラム名統一
df_2021 = df_2021.rename(columns={"商品": "品名"})
df_2022 = df_2022.rename(columns={"商品": "品名"})
df_2023 = df_2023.rename(columns={"商品": "品名"})
# df_2024 は既に「品名」なのでそのまま

# 結合
df_all = pd.concat([df_2021, df_2022, df_2023, df_2024], ignore_index=True)

# 日付をdatetime型に変換
df_all["伝票日付"] = pd.to_datetime(df_all["伝票日付"], errors='coerce')
df_all["正味重量"] = pd.to_numeric(df_all["正味重量"], errors='coerce')
df_all = df_all.dropna()

/tmp/ipykernel_788787/2293148294.py:15: DtypeWarning: Columns (68) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2023 = pd.read_csv(path_2023, encoding="utf-8")[["伝票日付", "正味重量", "商品"]]


In [ ]:
holiday_all = get_japanese_holidays(df_all["伝票日付"].min(), df_all["伝票日付"].max())

### 正規化

In [ ]:
import pandas as pd
import unicodedata
import re

# 揺れ吸収マスタ辞書（今後も随時拡張可）
replace_dict = {
    'ダンボ-ル': 'ダンボール',
    'スチ-ル': 'スチール',
    'トナー他': 'トナー',
    'GC軽鉄・スチ-ル類.': 'GC軽鉄・スチール類',
    'GC軽鉄・スチ-ル類': 'GC軽鉄・スチール類',
    'GC軽鉄・スチ-ル': 'GC軽鉄・スチール',
    'GAH鋼・鉄筋等': 'GA H鋼・鉄筋等',
    'フロン行程表': 'フロン行程管理票',
    'コンクリ-ト': 'コンクリート',
    'アスガラ': 'アスファルトがら',
}

# 半角カナ→全角カナ置換辞書
kana_replace_dict = {
    'ｿﾌｧｰ': 'ソファー',
    'ｶﾞﾗｽ': 'ガラス',
    'ﾋﾞﾝ': 'ビン',
    'ｶﾝ': 'カン',
}

def normalize_item_name(name):
    # ① 全角→半角正規化
    name = unicodedata.normalize("NFKC", name)
    
    # ② 半角カナ置換
    for key, val in kana_replace_dict.items():
        name = name.replace(key, val)
    
    # ③ 辞書置換
    for key, val in replace_dict.items():
        name = name.replace(key, val)
    
    # ④ 余計な空白除去（全角・半角・タブなど）
    name = re.sub(r'\s+', '', name)
    
    # ⑤ 長音記号統一（ハイフン→長音）
    name = name.replace('-', 'ー')
    
    return name

# 一括適用
df_all["品名"] = df_all["品名"].astype(str).apply(normalize_item_name)


df_all


,伝票日付,正味重量,品名
0,2021-01-04,1440.0,室内機
1,2021-01-04,1440.0,混合廃棄物A
2,2021-01-04,1440.0,室外機
3,2021-01-05,920.0,混合廃棄物A
4,2021-01-05,870.0,混合廃棄物A
...,...,...,...
122953,2024-03-31,970.0,混合廃棄物A
122954,2024-03-31,970.0,GC軽鉄・スチール類
122955,2024-03-31,580.0,混合廃棄物A
122956,2024-03-31,530.0,木くず


In [ ]:

from logic.factory_manage.predict_model_v4 import generate_features, run_fully_walkforward_pipeline
import pandas as pd
from datetime import timedelta

df_feat, df_pivot = generate_features(df_all,holidays=holiday_all)

print(f"date {df_all['伝票日付'].min()} to {df_all['伝票日付'].max()} with {len(df_all)} records")


date 2021-01-04 00:00:00 to 2024-03-31 00:00:00 with 122957 records


### 長期データにて予測

In [ ]:
pred_df = run_fully_walkforward_pipeline(
    df_feat,
    df_pivot,
)

🕒 Processing 2021-01-12 ...
  📦 Training for 混合廃棄物A
  📦 Training for 混合廃棄物B
🕒 Processing 2021-01-13 ...
  📦 Training for 混合廃棄物A
  📦 Training for 混合廃棄物B
🕒 Processing 2021-01-14 ...
  📦 Training for 混合廃棄物A
  📦 Training for 混合廃棄物B
🕒 Processing 2021-01-15 ...
  📦 Training for 混合廃棄物A
  📦 Training for 混合廃棄物B
🕒 Processing 2021-01-16 ...
  📦 Training for 混合廃棄物A
  📦 Training for 混合廃棄物B
🕒 Processing 2021-01-17 ...
  📦 Training for 混合廃棄物A
  📦 Training for 混合廃棄物B
🕒 Processing 2021-01-18 ...
  📦 Training for 混合廃棄物A
  📦 Training for 混合廃棄物B
🕒 Processing 2021-01-19 ...
  📦 Training for 混合廃棄物A
  📦 Training for 混合廃棄物B
🕒 Processing 2021-01-20 ...
  📦 Training for 混合廃棄物A
  📦 Training for 混合廃棄物B
🕒 Processing 2021-01-21 ...
  📦 Training for 混合廃棄物A
  📦 Training for 混合廃棄物B
🕒 Processing 2021-01-22 ...
  📦 Training for 混合廃棄物A
  📦 Training for 混合廃棄物B
🕒 Processing 2021-01-23 ...
  📦 Training for 混合廃棄物A
  📦 Training for 混合廃棄物B
🕒 Processing 2021-01-24 ...
  📦 Training for 混合廃棄物A
  📦 Training for 混合廃棄物B
🕒 Processing

### 最適な履歴長さ

In [ ]:
from logic.factory_manage.predict_model_v4 import find_optimal_history, train_final_model

In [ ]:
df_feat, df_pivot = generate_features(df_all, holiday_all)

In [ ]:
# 試したい履歴長リスト（例）
history_windows = [100, 200, 300, 400, 500, 700, 1000]

# 最適履歴長を探索
best_window = find_optimal_history(df_feat, df_pivot, history_windows)



===== 検証: 最新 100日履歴 =====
検証 R²=0.602, MAE=19,347kg

===== 検証: 最新 200日履歴 =====
検証 R²=0.621, MAE=18,583kg

===== 検証: 最新 300日履歴 =====
検証 R²=0.623, MAE=18,775kg

===== 検証: 最新 400日履歴 =====
検証 R²=0.574, MAE=20,026kg

===== 検証: 最新 500日履歴 =====
検証 R²=0.567, MAE=20,011kg

===== 検証: 最新 700日履歴 =====
検証 R²=0.578, MAE=19,897kg

===== 検証: 最新 1000日履歴 =====


KeyboardInterrupt: 

### 1日分将来予測

In [4]:
from logic.factory_manage.predict_model_v5 import walkforward_one_day_full_stage2


In [5]:

# 試したい履歴長リスト（例）
history_windows = [100, 200, 300, 400, 500, 700, 1000]
for num in history_windows:
    print(f"Testing history window: {num}")
    r2, mae = walkforward_one_day_full_stage2(df_feat, df_pivot, num)
    print(f"R2: {r2}, MAE: {mae}")


# r2, mae = walkforward_one_day_full_stage2(df_feat, df_pivot, history_window)


Testing history window: 100

===== 各日の予測結果 =====

===== 完全ウォークフォワード1日先評価 結果 =====
R² = 0.627, MAE = 13,663kg
R2: 0.6267040552065005, MAE: 13662.717673993126
Testing history window: 200

===== 各日の予測結果 =====

===== 完全ウォークフォワード1日先評価 結果 =====
R² = 0.670, MAE = 13,097kg
R2: 0.6704885178415823, MAE: 13096.505785807072
Testing history window: 300

===== 各日の予測結果 =====

===== 完全ウォークフォワード1日先評価 結果 =====
R² = 0.673, MAE = 12,879kg
R2: 0.6728608504666611, MAE: 12878.651596235037
Testing history window: 400

===== 各日の予測結果 =====

===== 完全ウォークフォワード1日先評価 結果 =====
R² = 0.723, MAE = 11,703kg
R2: 0.7231127524046694, MAE: 11702.709635277453
Testing history window: 500

===== 各日の予測結果 =====

===== 完全ウォークフォワード1日先評価 結果 =====
R² = 0.723, MAE = 11,703kg
R2: 0.7231127524046694, MAE: 11702.709635277453
Testing history window: 700

===== 各日の予測結果 =====

===== 完全ウォークフォワード1日先評価 結果 =====
R² = 0.723, MAE = 11,703kg
R2: 0.7231127524046694, MAE: 11702.709635277453
Testing history window: 1000

===== 各日の予測結果 =====

===== 完